In [1]:
!pip install -q transformers sentencepiece

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load NLLB translator model
model_id = "facebook/nllb-200-distilled-600M"
trans_tokenizer = AutoTokenizer.from_pretrained(model_id)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

2025-09-18 13:43:51.838862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758203032.041559      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758203032.097875      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import torch
from tqdm.auto import tqdm

# Define the separator-based splitting function
def split_instruction_and_example(text):
    for sep in ["Example:", "উদাহরণ:", "Exammple:", " উদাহরণ:", " উদাহরণঃ", "Ex:"]:
        if sep in text:
            parts = text.split(sep, 1)
            instruction = parts[0].strip()
            example = parts[1].strip()
            return instruction, example
    return text.strip(), ""  # no example found

# Translation function 
def translate_bn_to_en(text: str) -> str:
    inputs = trans_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    translated_ids = trans_model.generate(
        **inputs,
        forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),
        max_new_tokens=256
    )

    return trans_tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

# Process the dev.csv
df = pd.read_csv("/kaggle/input/blptask2data/dev_v2.csv")  # change the file path accordingly
assert {"id", "instruction", "test_list"}.issubset(df.columns)

# Process each instruction (split + translate)
final_prompts = []

for raw in tqdm(df["instruction"], desc="🔁 Translating instructions"):
    bangla_instr, example = split_instruction_and_example(raw)
    # print(bangla_instr, example)

    try:
        english_instr = translate_bn_to_en(bangla_instr)
    except Exception as e:
        english_instr = "[Translation Failed]"
        print("⚠️ Error:", e)

    # Combine again (optional: add separator for clarity)
    if example:
        full_prompt = f"{english_instr.strip()} Example:\n{example.strip()}"
    else:
        full_prompt = english_instr.strip()

    # print(full_prompt)
    final_prompts.append(full_prompt)
    # break

# for ls in final_prompts:
#     print(ls)

# Create a new DataFrame with id and translated_prompt
out_df = pd.DataFrame({
    "id": df["id"],
    "instruction": final_prompts
})

# Save to CSV
out_df.to_csv("dev_translated.csv", index=False)
print(" Saved translated file → dev_translated.csv")

🔁 Translating instructions:   0%|          | 0/400 [00:00<?, ?it/s]

✅ Saved translated file → dev_translated.csv
